# Running the model

Setup a `feisty` integration.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import yaml

import feisty

## Configure testcase to run

TODO: this is using the first column (which I believe is shallowest); want to add deepest column as well.
That will require refactoring the yaml file and also changing test_ds.

In [3]:
with open("matlab_first_timestep.yaml") as f:
    baselines = yaml.safe_load(f)


def compare(baselines_dict, da_or_ds, full_table=True, markdown_formatting=False):
    if markdown_formatting:
        if "dimname" in baselines_dict:
            print(f"| {baselines_dict['dimname']} | Matlab Value | Python Value | Rel Err |")
        else:
            print(f"|  | Matlab Value | Python Value | Rel Err |")
        print("| --- | --- | --- | --- |")
        seps = ["| ", " | ", " | ", " |"]
    else:
        seps = ["", ": ", ", ", ""]
    if "dimname" in baselines_dict:
        all_vars = da_or_ds[baselines_dict["dimname"]].data
    else:
        all_vars = baselines_dict["values"]
    for var in all_vars:
        if "dimname" in baselines_dict:
            da = da_or_ds
            sel_dict = {baselines_dict["dimname"]: var}
        else:
            da = da_or_ds[var]
            sel_dict = {}
        matlab_val = baselines_dict["values"][var]
        py_val = da.sel(sel_dict).data
        if matlab_val != 0:
            rel_err = (py_val - matlab_val) / matlab_val
            if rel_err < 0:
                rel_err = -rel_err
        else:
            if py_val == 0:
                rel_err = 0
            elif py_val > 0:
                rel_err = np.inf
            elif py_val < 0:
                rel_err = -np.inf
        if full_table or rel_err > 1e-2:
            print(
                f"{seps[0]}{var}{seps[1]}{matlab_val:10.4e}{seps[2]}"
                + f"{py_val:10.4e}{seps[2]}{rel_err:10.4e}{seps[-1]}"
            )

In [4]:
matlab_script = "test_case"

if matlab_script == "test_case":
    testcase = feisty.driver.simulate_testcase("tanh_shelf", "cyclic")
else:
    raise ValueError(f"unknown matlab_script '{matlab_script}'")

In [5]:
compare(
    baselines[matlab_script]["day1"]["forcing"],
    testcase.forcing.isel(time=0, X=0, zooplankton=0),
    markdown_formatting=True,
)

|  | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| T_pelagic | 1.8200e+01 | 1.8200e+01 | 0.0000e+00 |
| T_bottom | 4.2000e+00 | 4.2000e+00 | 0.0000e+00 |
| zooC | 3.3287e+00 | 3.3287e+00 | 0.0000e+00 |
| poc_flux_bottom | 2.7847e-02 | 2.7847e-02 | 1.2459e-16 |
| zoo_mort | 7.7564e-01 | 7.7564e-01 | 0.0000e+00 |


## Run the model


In [6]:
%%time

nsteps = 365
make_plot = False
testcase.run(nsteps)
testcase.ds

CPU times: user 1min 22s, sys: 34.8 ms, total: 1min 22s
Wall time: 1min 29s


<xarray.Dataset>
Dimensions:                    (X: 22, group: 9, time: 365, fish: 8, feeding_link: 19)
Coordinates:
  * X                          (X) float64 -0.5 -0.2381 0.02381 ... 4.738 5.0
  * group                      (group) <U12 'Sf' 'Sp' ... 'Ld' 'benthic_prey'
  * fish                       (fish) <U2 'Sf' 'Sp' 'Sd' 'Mf' ... 'Md' 'Lp' 'Ld'
  * feeding_link               (feeding_link) <U15 'Sf_Zoo' ... 'Ld_benthic_p...
    predator                   (feeding_link) <U2 'Sf' 'Sp' 'Sd' ... 'Ld' 'Ld'
    prey                       (feeding_link) <U12 'Zoo' ... 'benthic_prey'
Dimensions without coordinates: time
Data variables: (12/16)
    biomass                    (time, group, X) float64 9.999e-06 ... 0.05542
    T_habitat                  (time, fish, X) float64 18.2 18.2 ... 18.2 18.2
    ingestion_rate             (time, fish, X) float64 0.2152 ... 2.234e-05
    predation_flux             (time, fish, X) float64 5.737e-12 ... 0.0
    predation_rate             (time, fish, X) float64 5.737e-07 ... 0.0
    metabolism_rate            (time, fish, X) float64 0.04296 ... 0.00488
    ...                         ...
    recruitment_flux           (time, fish, X) float64 1.453e-09 ... 0.0
    fish_catch_rate            (time, fish, X) float64 0.0 0.0 ... 0.0008219
    encounter_rate_link        (time, feeding_link, X) float64 2.288 ... 0.0
    encounter_rate_total       (time, fish, X) float64 2.288 2.288 ... 2.239e-05
    consumption_rate_max_pred  (time, fish, X) float64 0.2375 0.2375 ... 0.01062
    consumption_rate_link      (time, feeding_link, X) float64 0.2152 ... 0.0

In [7]:
check_key = f"day{nsteps}"

if nsteps > 1:
    if make_plot:
        for group in test_ds.group.data:
            testcase.ds.biomass.sel(group=group).isel(X=0).plot()
        plt.ylim([1e-6, 1])
        plt.yscale("log")
        plt.legend(test_ds.group.data)

test_ds = testcase.ds.isel(X=0, time=-1)

In [8]:
compare(
    baselines[matlab_script][check_key]["biomass"],
    test_ds["biomass"],
    markdown_formatting=True,
)

| group | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 1.7696e-05 | 1.7696e-05 | 0.0000e+00 |
| Sp | 9.3653e-06 | 9.3653e-06 | 0.0000e+00 |
| Sd | 9.4481e-06 | 9.4481e-06 | 0.0000e+00 |
| Mf | 4.0856e-04 | 4.0856e-04 | 1.3269e-16 |
| Mp | 3.1926e-04 | 3.1926e-04 | 0.0000e+00 |
| Md | 3.1755e-04 | 3.1755e-04 | 1.7071e-16 |
| Lp | 4.7386e-04 | 4.7386e-04 | 0.0000e+00 |
| Ld | 2.2687e-04 | 2.2687e-04 | 7.1683e-16 |
| benthic_prey | 6.3646e-01 | 6.3646e-01 | 0.0000e+00 |
